In [1]:
#Exercise1 
import numpy as np
from sklearn.model_selection import KFold

X = np.array(np.arange(1, 21).reshape(10, -1))
y = np.array(np.arange(1, 11))
kf = KFold(n_splits=5)

for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
    print(f"Fold: {fold}")
    print(f"TRAIN: {train_index} TEST: {test_index}\n")
    



Fold: 1
TRAIN: [2 3 4 5 6 7 8 9] TEST: [0 1]

Fold: 2
TRAIN: [0 1 4 5 6 7 8 9] TEST: [2 3]

Fold: 3
TRAIN: [0 1 2 3 6 7 8 9] TEST: [4 5]

Fold: 4
TRAIN: [0 1 2 3 4 5 8 9] TEST: [6 7]

Fold: 5
TRAIN: [0 1 2 3 4 5 6 7] TEST: [8 9]



In [2]:
#Exercise2
# imports
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# data
housing = fetch_california_housing()
X, y = housing['data'], housing['target']

# split data train test
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.1,
                                                    shuffle=True,
                                                    random_state=43)

# pipeline
pipeline = [('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler()),
            ('lr', LinearRegression())]
pipe = Pipeline(pipeline)

from sklearn.model_selection import cross_validate

# Cross-validate with 10 folds
cv_results = cross_validate(pipe, X_train, y_train, cv=10, scoring='r2')

scores = cv_results['test_score']
print("Scores on validation sets:")
print(scores)

mean_score = scores.mean()
print("\nMean of scores on validation sets:")
print(mean_score)

std_score = scores.std()
print("\nStandard deviation of scores on validation sets:")
print(std_score)

Scores on validation sets:
[0.62433594 0.61648956 0.62486602 0.59891024 0.59284295 0.61307055
 0.54630341 0.60742976 0.60014575 0.59574508]

Mean of scores on validation sets:
0.6020139252674299

Standard deviation of scores on validation sets:
0.02149838227734666


In [ ]:
#Exercise 3
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# data
housing = fetch_california_housing()
X, y = housing['data'], housing['target']
# split data train test
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.1,
                                                    shuffle=True,
                                                    random_state=43)

param_grid = {
    'max_depth': [1, 5, 10, 15, 20],  # Tree depth: deeper trees can overfit
    'n_estimators': [10, 50, 100]     # Number of trees: more trees reduce variance but increase time
}

rf = RandomForestRegressor(random_state=43)  

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    scoring='neg_mean_squared_error',  # Negative MSE (since GridSearch maximizes)
    n_jobs=-1  # Use all CPUs
)

grid_search.fit(X_train, y_train)



In [ ]:
best_estimator = grid_search.best_estimator_
print("Best parameters:")
print(grid_search.best_params_)

print("\nBest score on validation sets (neg MSE):")
print(grid_search.best_score_)

print("\nCV results:")
print(grid_search.cv_results_)

In [ ]:
test_score = best_estimator.score(X_test, y_test)
print("\nScore on test set (R²):")
print(test_score)